In [5]:
import numpy as np
import pandas as pd

from pmdarima.arima import auto_arima

from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

In [7]:
SPY_df = pd.read_csv ('SPY.csv')

In [12]:
#Item 1
SPY_df[['Date','Close']].head()

,Date,Close
0,2000-01-03,145.4375
1,2000-01-04,139.7500
2,2000-01-05,140.0000
3,2000-01-06,137.7500
4,2000-01-07,145.7500


In [13]:
#Item 2
SPY_df[['Date','High','Low','Volume','Close']].head()

,Date,High,Low,Volume,Close
0,2000-01-03,148.25000,143.875000,8164300,145.4375
1,2000-01-04,144.06250,139.640625,8089800,139.7500
2,2000-01-05,141.53125,137.250000,12177900,140.0000
3,2000-01-06,141.50000,137.750000,6227200,137.7500
4,2000-01-07,145.75000,140.062500,8066500,145.7500


In [286]:
train = SPY_df.head(3876)
test = SPY_df.tail(1660)

train_tune = train.head(3101)
train_valid = train.tail(776)

In [201]:
arma_model = auto_arima(train['Close'], out_of_sample_size = 776)

In [202]:
arma_model

ARIMA(order=(1, 1, 1), out_of_sample_size=776, scoring_args={},
      suppress_warnings=True, with_intercept=False)

In [203]:
arma_full_model = auto_arima(train['Close'],p=1,d=1,q=1)

In [392]:
arma_pred = []
for i in range(20):
    arma_full_model = auto_arima(SPY_df.iloc[(0+i):(3876+i)]['Close'],p=1,d=1,q=1)
    arma_pred.append(arma_full_model.predict(1))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [405]:
np.mean((np.array(arma_pred)[0] - test['Close'][:20])**2)

C:\Users\13127\AppData\Local\Temp\ipykernel_4404\2726435.py:1: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  np.mean((np.array(arma_pred)[0] - test['Close'][:20])**2)


4.256415205088304

In [441]:
LSTM_tries = [[1,[128]],[2,[64,64]]]

In [442]:
#Item 3
def auto_LSTM(train_tune_df, train_valid_df, mode='univariate', window_size = [256,512], \
             LSTM_setup = LSTM_tries, epochs_size = 2, h = 1):
    
    def windowing(series, window):
        X, Y = list(), list()
        for i in range(len(series)):
            take_rng = i + window
            if take_rng > len(series) - 1:
                break
            x, y = series[i:take_rng], series[take_rng]
            X.append(x)
            Y.append(y)

        return np.array(X), np.array(Y)
    
    def standardize(series,mean=None,sd=None):
        if (mean == None) and (sd == None):
            mean = np.mean(series)
            sd = (np.var(series))**0.5
            return (series - mean)/sd,mean,sd
        
        else:
            return (series - mean)/sd,mean,sd

    if mode == 'univariate':
        
        res_dict = dict()
        
        for window in window_size:
            
            train, train_mu, train_sigma = standardize(train_tune_df.reset_index()['Close'])
            val, _ , _ = standardize(train_valid_df.reset_index()['Close'], mean = train_mu, sd = train_sigma)
            X, Y = sampling(train, window)
            X_val, Y_val = sampling(val, window)
            x_raw_train, y_raw_train = sampling(train_tune_df.reset_index()['Close'],window)
            x_raw_val, y_raw_val = sampling(train_valid_df.reset_index()['Close'],window)
            
            for set_up in LSTM_setup:
                num_layers = set_up[0]
                num_neurons = set_up[1]
                
                curr_model = keras.Sequential()
                
                if num_layers == 1:
                    curr_model.add(layers.LSTM(num_neurons[0], input_shape=(window, h),return_sequences=False))

                else:
                    for layer in range(num_layers - 1):
                        if layer == 0:
                            curr_model.add(layers.LSTM(num_neurons[layer], input_shape=(window, h),return_sequences=True))
                        else:
                            curr_model.add(layers.LSTM(num_neurons[layer], return_sequences=True))
                    
                    curr_model.add(layers.LSTM(num_neurons[-1], return_sequences=False))

                curr_model.add(layers.Dense(1))
                curr_model.compile(optimizer='adam', loss='mse', metrics=['mean_squared_error'])
                print("training:" + str(set_up) + ' window_size=' + str(window))
                curr_model.fit(X, Y, epochs=epochs_size)
                
                train_pred = (curr_model.predict(X) * train_sigma) + train_mu
                train_score = np.mean((train_pred - y_raw_train)**2)
                
                val_pred = (curr_model.predict(X_val) * train_sigma) + train_mu
                val_score = np.mean((val_pred - y_raw_val)**2)
                
                
                res_dict[str(set_up) + ' window_size=' + str(window)] = val_score
                print(str(set_up) + ' window_size=' + str(window) + " train_score: " + str(train_score))
                print(str(set_up) + ' window_size=' + str(window) + " val_score: " + str(val_score))
                curr_model = None
    
    return min(res_dict, key=res_dict.get)


In [443]:
#Item 4
res = auto_LSTM(train_tune, train_valid)
print(res)

def fit_LSTM(train_df,num_layers, num_neurons, window, mode = 'univariate',h = 1, \
             epochs_size = 2):
    
    if mode == 'univariate':
        train, train_mu, train_sigma = standardize(train_df.reset_index()['Close'])
        X, Y = sampling(train, window)

        curr_model = keras.Sequential()

        if num_layers == 1:
            curr_model.add(layers.LSTM(num_neurons[0], input_shape=(window, h),return_sequences=False))

        else:
            for layer in range(num_layers - 1):
                if layer == 0:
                    curr_model.add(layers.LSTM(num_neurons[layer], input_shape=(window, h),return_sequences=True))
                else:
                    curr_model.add(layers.LSTM(num_neurons[layer], return_sequences=True))

            curr_model.add(layers.LSTM(num_neurons[-1], return_sequences=False))

        curr_model.add(layers.Dense(1))
        curr_model.compile(optimizer='adam', loss='mse', metrics=['mean_squared_error'])
        curr_model.fit(X, Y, epochs=epochs_size)
    
    return [curr_model,train_mu,train_sigma]

LSTM_1 = fit_LSTM(train,1,[128],512, epochs_size = 2)

LSTM_pred = []
for i in range(20):
    input_data = (SPY_df.iloc[(0+i):(3876+i)]['Close'].tail(512)-LSTM_1[1])/LSTM_1[2]
    LSTM_pred.append(float((LSTM_1[0].predict([list(input_data)])*LSTM_1[2])+LSTM_1[1]))

np.mean((np.array(LSTM_pred) - test['Close'][:20])**2)

plt.plot(train['Close'].loc[train.shape[0]-100:train.shape[0]], color='blue')
plt.plot(test['Close'][:20], color='blue', alpha=0.2)
plt.plot([3876 + i for i in range(20)], arma_pred, color = 'red', linestyle = "--",label='ARMA')
plt.plot([3876 + i for i in range(20)], LSTM_pred, color = 'green', linestyle = "--",label='LSTM')
plt.xlabel('t')
plt.ylabel('SPY Index Close')
plt.legend(loc="upper left")

training:[1, [128]] window_size=256
Epoch 1/2
89/89 [==============================] - 122s 1s/step - loss: 0.0606 - mean_squared_error: 0.0606
Epoch 2/2
17/17 [==============================] - 10s 566ms/step
[1, [128]] window_size=256 train_score: 595.3025874124362
[1, [128]] window_size=256 val_score: 421.45541729874066
training:[2, [64, 64]] window_size=256
Epoch 1/2
89/89 [==============================] - 106s 1s/step - loss: 0.0689 - mean_squared_error: 0.0689
Epoch 2/2
17/17 [==============================] - 7s 387ms/step
[2, [64, 64]] window_size=256 train_score: 616.1528403197798
[2, [64, 64]] window_size=256 val_score: 510.8949920261972
training:[1, [128]] window_size=512
Epoch 1/2
81/81 [==============================] - 179s 2s/step - loss: 0.0706 - mean_squared_error: 0.0706
Epoch 2/2
9/9 [==============================] - 11s 1s/step
[1, [128]] window_size=512 train_score: 667.9483701814843
[1, [128]] window_size=512 val_score: 334.055982082147
training:[2, [64, 64]] wi